Notebook to study characteristic frequencies of different QPO models applied to the supermassive black hole 2MASXJ12571076+2724177

Known parameters:
* $M = 10^{6.3} M_\odot$
* $\\nu = 3.3*10^{-5}$ HZ
* $d = 90\\, Mpc&

let's look for a range of frequencies and masses around these parameters